## ДЗ 9

Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.
Mobile Games AB Testing with Cookie Cats

In [1]:
import pandas as pd

!pip install bootstrapped

import bootstrapped.bootstrap as bs
import bootstrapped.compare_functions as bs_compare
import bootstrapped.stats_functions as bs_stats

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
df = pd.read_excel('AB_results.xlsx')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1


In [3]:
df.drop_duplicates()
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


In [4]:
session_counts = df['userid'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'{multi_users} пользователей появляется в датасете несколько раз')

0 пользователей появляется в датасете несколько раз


Это значит никто не попал одновременно в оба сплита.

In [5]:
df_ab = df.groupby("version").agg({"userid":"count", "retention_1":"mean","retention_7":"mean"})
df_ab

,userid,retention_1,retention_7
version,,,
gate_30,44700,0.448188,0.190201
gate_40,45489,0.442283,0.182000


Сплиты по размеру сравнимы.
Дневной ретеншн почти не отличается, недельный отличается на ~4% в худшую сторону.

In [6]:
gate_30 = df.loc[df['version'] == 'gate_30']
gate_40 = df.loc[df['version'] == 'gate_40']

g3_1 = gate_30['retention_1'].values
g3_7 = gate_30['retention_7'].values
g4_1 = gate_40['retention_1'].values
g4_7 = gate_40['retention_7'].values

In [7]:
b_1 = bs.bootstrap_ab(g3_1, g4_1, stat_func=bs_stats.mean, compare_func=bs_compare.difference, alpha=0.05, num_iterations=10000)

print(b_1.lower_bound, b_1.upper_bound)

-0.0005856803495970404 0.012466832862166996


Для дневного ретеншна подтверждается нулевая гипотеза.

In [8]:
b_7 = bs.bootstrap_ab(g3_7, g4_7, stat_func=bs_stats.mean, compare_func=bs_compare.difference, alpha=0.05, num_iterations=10000)

print(b_7.lower_bound, b_7.upper_bound)

0.003041891387268023 0.013179073346296087


Для недельного ретеншна нулевая гипотеза отвергается, то есть наблюдаются статистически значимые различия.
Но учитывая, что различия эти в худшую сторону, то рекоммендация gate_30 на gate_40 **не менять**.